In [1]:
import numpy as np
from numpy import array
from tensorflow.keras.models import Model

from spektral.data import Dataset,Graph
from spektral.transforms.normalize_adj import NormalizeAdj

#DataFrame
import pandas as pd

from scipy.sparse import csr_matrix
#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool

import tensorflow as tf
from spektral.models import GeneralGNN

from spektral.data import DisjointLoader

from tensorflow.keras.optimizers import Adam

import numpy as np
import tensorflow as tf
from ogb.graphproppred import GraphPropPredDataset, Evaluator
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import OGB
from spektral.layers import ECCConv, GlobalSumPool

Using backend: pytorch


In [2]:
class MyDataset(Dataset):
    
    def __init__(self, n_samples, **kwargs):
        self.n_samples = n_samples
        super().__init__(**kwargs)

    def read(self):
        
        def make_graph():
           
            proteine_path = "../Dati/proteine.csv"
            proteine = pd.read_csv(proteine_path, sep = ',')
            for filename in os.listdir("../Dati/Grafi/"):  
                with open(os.path.join("../Dati/Grafi/", filename), 'r', encoding='windows-1252') as f:
                    id_p = filename[0:4]
                    if (filename != ".DS_Store") and (id_p not in id_p_e):
                        id_p_e.append(id_p)
                        data = np.load("../Dati/Grafi/{}.npz".format(id_p), allow_pickle=True)
                        
                        #print(id_p, id_p_e)
                        
                        return Graph(x= data['x'], a= csr_matrix(data['a'].all()), e = None, y= data['y'])

        # We must return a list of Graph objects
        return [make_graph() for _ in range(self.n_samples)]

In [3]:
# Creazione GNN

In [4]:
dataset = MyDataset(100)
dataset

MyDataset(n_graphs=100)

In [5]:
for g in dataset:
    print(g, g.y)

Graph(n_nodes=179, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=126, n_node_features=1, n_edge_features=None, n_labels=1) 0
Graph(n_nodes=129, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=178, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=128, n_node_features=1, n_edge_features=None, n_labels=1) 0
Graph(n_nodes=178, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=110, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=102, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=126, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=178, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=135, n_node_features=1, n_edge_features=None, n_labels=1) 0
Graph(n_nodes=132, n_node_features=1, n_edge_features=None, n_labels=1) 1
Graph(n_nodes=123, n_node_features=1, n_edge_features=None, n_labels=1) 0
Graph(n_nodes=131, n_node_features=1, 

In [8]:
class MyFirstGNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.1)
        self.dense = Dense(n_labels, 'softmax')

    def call(self, inputs):
        #print(inputs)
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

1

In [14]:
model = MyFirstGNN(10, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

In [15]:
# Train/test split
np.random.shuffle(dataset)
split = int(0.8 * len(dataset))
dataset_train, dataset_test = dataset[:split], dataset[split:]

In [16]:
from spektral.data import BatchLoader
from spektral.data import DisjointLoader
loader = BatchLoader(dataset_train, batch_size=10)
#loader = DisjointLoader(dataset, batch_size=10)

In [17]:
loader.steps_per_epoch

8

In [18]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10
8/8 [==============================] - 1s 8ms/step - loss: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 9ms/step - loss: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 0s 4ms/step - loss: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 0s 5ms/step - loss: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 0s 6ms/step - loss: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 0s 7ms/step - loss: 0.0000e+00
Epoch 8/10
8/8 [==============================] - 0s 6ms/step - loss: 0.0000e+00
Epoch 9/10
8/8 [==============================] - 0s 6ms/step - loss: 0.0000e+00
Epoch 10/10
8/8 [==============================] - 0s 6ms/step - loss: 0.0000e+00


In [19]:
loader = BatchLoader(dataset_test, batch_size=32)

In [20]:
loss = model.evaluate(loader.load(), steps=loader.steps_per_epoch)

print('Test loss: {}'.format(loss))

1/1 [==============================] - 0s 141ms/step - loss: 0.0000e+00
Test loss: 0.0
